# Lectura de archivos 


Semana Tec

Profesores: Luz Eunice Angeles /José Ignacio Treviño

Qué aprenderemos: 

- Subir archivos desde el sistema de archivos local 
- Descargar archivos en el slistema de archivos local 
- Acceder a google drive 
  - Activarlo de en el entorno de ejecución
  - Usar una API : PyDrive
- Crear y leer datos de google sheets


## Subir archivos desde el sistema de archivos local

<code>files.upload</code> es usado para cargar los archivos y conocer el diccionario de los archivos que se cargaron.
El diccionario está codificado según el nombre de archivo y los valores son los datos que se subieron.

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ejemploClase.csv to ejemploClase.csv
User uploaded file "ejemploClase.csv" with length 38 bytes


## Descargar archivos en el sistema de archivos local

<code>files.download</code> invocará una descarga del archivo desde el navegador a tu computadora local.


In [4]:
from google.colab import files

with open('ejemploST.txt', 'w') as f:
  f.write('Ejemplo de la clase de semana tec')

files.download('ejemploST.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
ls

ejemploClase.csv  ejemploST.txt  sample_data/


# Google Drive

Puedes acceder a archivos en Drive de varias maneras, incluidas las siguientes:
- Activación de Google Drive en la máquina virtual del entorno de ejecución
- Usar un wrapper alrededor de la API, como <a href="https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html">PyDrive</a>
- Usar la <a href="https://developers.google.com/drive/v3/web/about-sdk">API nativa de REST</a>



Puedes ver ejemplos de cada caso más abajo.

## Activar Google Drive de forma local

El ejemplo siguiente muestra cómo activar Google Drive en el entorno de ejecución mediante un código de autorización y cómo escribir y leer archivos allí. Una vez ejecutado, podrás ver el archivo nuevo &#40;<code>foo.txt</code>&#41; en <a href="https://drive.google.com/">https://drive.google.com/</a>.

Ten en cuenta que solo se puede leer, escribir o mover archivos. 

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
with open('/content/drive/My Drive/SemanaTec/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/SemanaTec/foo.txt

Hello Google Drive!

## PyDrive

En los ejemplos que aparecen a continuación, se muestran la autenticación y la carga y descarga de archivos mediante PyDrive. Puedes encontrar más ejemplos en la <a href="https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html">documentación de PyDrive</a>.

In [11]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autentica y crea el cliente de PyDrive.


In [12]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Lista de ids de cada carpeta que tenemos en el google drive

In [13]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:       
    print ('title: %s, id: %s' % (file1['title'], file1['id']))


title: 3.1 - CargaDatos  archivos locales, Drive, Hojas de calculo.ipynb, id: 15k0OaCo0ZjoCXeG4UeTyoIhHqFwIfY6q
title: 2.3 - Actualizar repositorio.ipynb, id: 120xMOI2cNw5Id-lEnZ5Q-exjy1hjTql6
title: Equipo1TC1002S, id: 1FVYNnS7MUH6eP800v8L1iXfO8WrflaaL
title: 2.1 - Básicos de Unix y GIT.ipynb, id: 10-9ADIV1jE1DNGjhYnXL_SvjuWwgNjct
title: SemanaTec, id: 1xwp4q5abFpKFxlASR1EFSB0_rJKrQqXG
title: Formato para coevaluación. Posible tema de investigación.docx, id: 1O8QQc0bqPhl9jKyblUpLDgv179_RDf2e
title: videoFinalNT2002B, id: 1w8JOJLCbkOrGFW9fLlIN7zKRg0ygnwPs6gm6ZDJELn4
title: Documentación y focalización de un problema. Formato.docx, id: 19IRHQf3ZaRO05wRief9MihPqB2SrRFTG
title: Funcionalizacion de biomoleculas NT2002B, id: 1ptnHt8nZnjxMdfSoeLzEE2ragUkgR7SfJDrG_DtnzM4
title: Jesús Riestra. Cuestionario.docx, id: 1VgIjKF5RluPEEmczHThwbi6_lD68wutT
title: servicio social, id: 15o_TvQm58xH9aVuyq-xf049kUu_MDAt7
title: Anexos, id: 14FWJT3l4qXYeuwWExne-KTlBGDUuW-FW
title: presupuest, id: 1F

In [14]:
fid= '1xwp4q5abFpKFxlASR1EFSB0_rJKrQqXG'

Crea y sube un archivo de texto.


In [15]:
uploaded = drive.CreateFile({'title': 'Sample EJEMPLOconPydrive.txt','parents':[{"id":fid}]})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1eCeZ7zVD9CVYMfByE6YErJLAtcnGD6Yr


Carga un archivo por su ID y luego imprime su contenido.


In [16]:
uploaded.get('id')

'1eCeZ7zVD9CVYMfByE6YErJLAtcnGD6Yr'

In [17]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


# Hojas de cálculo de Google

Los ejemplos de abajo usan la biblioteca de código abierto <a href="https://github.com/burnash/gspread"><code>gspread</code></a> para interactuar con Hojas de cálculo de Google.

Primero, instala el paquete usando <code>pip</code>.

In [18]:
!pip install --upgrade --quiet gspread

Importa la biblioteca, autentica y crea la interfaz para Hojas de cálculo.

In [19]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

A continuación, verás un pequeño conjunto de ejemplos de <code>gspread</code>. Puedes encontrar más ejemplos en la <a href="https://github.com/burnash/gspread#more-examples">página de GitHub de <code>gspread</code></a>.

## Crear una hoja nueva con datos de Python

In [20]:
sh = gc.create('My cool spreadsheet xdd')

Después de ejecutar la celda anterior, se mostrará una nueva hoja de cálculo con el nombre "My cool spreadsheet" en <a href="https://sheets.google.com/">https://sheets.google.com</a>.

Abre la hoja nueva y agrega datos aleatorios.

In [21]:
worksheet = gc.open('My cool spreadsheet xdd').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1aACq_kLW4KsHmQQuaxIhjFnmjCJP4dxrUNA2GPQdwnE',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': "'Hoja 1'!A1:C2",
 'updatedRows': 2}

## Descargar datos de una hoja de cálculo en Python como un DataFrame de Pandas

Vuelve a leer los datos aleatorios que insertamos anteriormente y convierte el resultado en un <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">DataFrame de Pandas</a>.

In [22]:
worksheet = gc.open('My cool spreadsheet xdd').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print('******************Lista de datos*********************')
print(rows)

import pandas as pd
dataset=pd.DataFrame.from_records(rows)
print('******************Data Frame*********************')
print(dataset)

******************Lista de datos*********************
[['1', '4', '6'], ['6', '7', '6']]
******************Data Frame*********************
   0  1  2
0  1  4  6
1  6  7  6
